In [ ]:
# !pip3 install preprocessing
!pip3 install -U gensim

In [ ]:
# !git clone https://github.com/rwalk/gsdmm.git

In [77]:
from gsdmm import MovieGroupProcess

import pickle
import matplotlib as plt
import pandas as pd
import numpy as np
import ast

In [78]:
import json
import pandas as pd
import csv
import os
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
from termcolor import colored
from gensim import corpora

# Topic Modeling 

## Data selection

In [3]:
with open('total_text.txt', encoding="utf8") as fopen:
    data = fopen.read().split('\n')[:-1]
len(data)

5657

In [4]:
data = pd.DataFrame(data, columns =['full_text'])

In [5]:
data.shape

(5657, 1)

In [6]:
data

,full_text
0,Starving plead help runs wild video based
1,People dropping like flies.Black death happeni...
2,Humor best medicine fight
3,footage Wuhan doctor mental breakdown
4,Wuhan began blocking major roads early hours T...
...,...
5652,china news ’ show Another body found mall wuha...
5653,Glad people finally caught coronoavirus WuhanC...
5654,Coronavirus impacts U.S. stock markets Dow Dro...
5655,tomgwilliam3 Realisation slowly dawns poor Bar...


In [7]:
def makelist(lst): 
    return [[el] for el in lst]

# Cleaning

#### Adding TQDM 

In [8]:
# from langdetect import detect
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

<ipython-input-8-8b96c5e442fc>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


C:\Users\User\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


#### Tokenization

In [9]:
from nltk.tokenize import sent_tokenize

data['sentences'] = data.full_text.progress_map(sent_tokenize)
data['sentences'].head(1).tolist()[0][:3] # Print the first 3 sentences of the 1st article

['Starving plead help runs wild video based']

In [10]:
from nltk.tokenize import word_tokenize

data['tokens_sentences'] = data['sentences'].progress_map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])
print(data['tokens_sentences'].head(1).tolist()[0][:3])


[['Starving', 'plead', 'help', 'runs', 'wild', 'video', 'based']]


In [11]:
data['tokens_sentences']

0       [[Starving, plead, help, runs, wild, video, ba...
1       [[People, dropping, like, flies.Black, death, ...
2                        [[Humor, best, medicine, fight]]
3           [[footage, Wuhan, doctor, mental, breakdown]]
4       [[Wuhan, began, blocking, major, roads, early,...
                              ...                        
5652    [[china, news, ’, show, Another, body, found, ...
5653    [[Glad, people, finally, caught, coronoavirus,...
5654    [[Coronavirus, impacts, U.S., stock, markets, ...
5655    [[tomgwilliam3, Realisation, slowly, dawns, po...
5656    [[Man, coronavirus, discharged, Toronto, hospi...
Name: tokens_sentences, Length: 5657, dtype: object

In [12]:
data['tokens_sentences'] = [[[w.lower() for w in tokens] for tokens in sentences] for sentences in data['tokens_sentences']]

tokens_listed = [[[print(w) for w in tokens] for tokens in sentences] for sentences in data['tokens_sentences']]

starving
plead
help
runs
wild
video
based
people
dropping
like
flies.black
death
happening
wuhan
right
humor
best
medicine
fight
footage
wuhan
doctor
mental
breakdown
wuhan
began
blocking
major
roads
early
hours
thursday
morning
large
number
people
wished
leave
wuhan
closing
city
unable
there
large
number
armed
police
hankou
railway
station
wuhan
wuhan
tianhe
airport
exception
what
's
h
thanks
constant
updates
colleagues
experts
it
amazing
follow
write
learn
ongoing
outbreak
following
experts
``
snake
''
probable
wildlife
animal
reservoir
wuhan
coronavirus
cathay
pacific
staff
can
wear
face
masks
first
case
wuhan
coronavirus
u.s.
found
north
seattle
january
gematria
broadcast
going
live
minutes
backup
channelnewsonthego
talking
latest
outbreak
everyone
working
vaccine
going
live
minutes
backup
channelnewsonthego
talking
latest
outbreak
when
u
find
son
's
st
sure
frontline
medical
stuff
working
day
night
horrible
environment
another
video
footage
taken
reveals
terrifying
situation
detec

In [13]:
# Remove punctuation from each word
puncts = str.maketrans('', '', string.punctuation)
data['tokens_sentences'] = [[[w.translate(puncts) for w in tokens] for tokens in sentences] for sentences in data['tokens_sentences']]

In [14]:
# filter out short tokens
data['tokens_sentences'] = [[[w for w in tokens if len(w) > 2] for tokens in sentences] for sentences in data['tokens_sentences']]

#### Regrouping tokens and removing stop words

In [15]:
#Remove stopwords
from nltk.corpus import stopwords
stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'should', 'would', 'can','could']
stopwords_other = ['wuhan', 'china', 'wuhanvirus', 'one', 'two', 'corona', 'rt', 'https', 'http', 'coronovirus', 'coronoavirus', 'image', 'coronavirus', 'covid', 'die', 'thing', 'time', 'pandemic', 'de', 'en', 'caption', 'also', 'copyright', 'something']
my_stopwords = stopwords.words('English') + stopwords_verbs + stopwords_other

In [16]:
from itertools import chain # to flatten list of sentences of tokens into list of tokens

data['tokens_without_sw'] = data['tokens_sentences'].map(lambda sentences: list(chain.from_iterable(sentences)))
data['tokens_without_sw'] = data['tokens_without_sw'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>1])

In [17]:
data['tokens_without_sw'] = makelist(data['tokens_without_sw'])
data['tokens_without_sw'].head(1).tolist()[0][:30]

[['starving', 'plead', 'help', 'runs', 'wild', 'video', 'based']]

In [18]:
#Remove quotes
data['tokens_without_sw'] = [[[re.sub("\'","",word) for word in tokens] for tokens in sentences] for sentences in data['tokens_without_sw']]
data['tokens'] = data['tokens_without_sw'].map(lambda sentences: list(chain.from_iterable(sentences)))

#### Prepare bi-grams and tri-grams

In [19]:
import gensim
from gensim.models import Phrases

stop_words = stopwords.words('english')
stop_words.extend(['RT'])

In [20]:
tokens = data['tokens'].tolist()

# Build the bigram and trigram models
bigram = Phrases(tokens)#, min_count=5, threshold=10, common_terms=stop_words)  # higher threshold fewer phrases.
trigram = Phrases(bigram[tokens], min_count=1)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

trigram_mod

In [21]:
# tokens = data['tokens'].tolist()
# bigram_model = Phrases(tokens)
# trigram_model = Phrases(bigram_model[tokens], min_count=1)
data['tokens'] = list(bigram[tokens])
data['tokens'] = makelist(data['tokens'])
data['tokens']

0        [[starving_plead, help_runs, wild_video, based]]
1       [[people_dropping, fliesblack, death, happenin...
2                        [[humor, best, medicine, fight]]
3                   [[footage, doctor, mental_breakdown]]
4       [[began, blocking, major, roads, early, hours,...
                              ...                        
5652    [[news, show, another, body, found, mall, wuha...
5653    [[glad, people, finally, caught, wuhancoronavi...
5654    [[impacts, stock, markets, dow, drops, points,...
5655    [[realisation, slowly, dawns, poor, barry, fac...
5656    [[man, discharged, toronto, hospital, novelcor...
Name: tokens, Length: 5657, dtype: object

#### Lemmatizing with POS tagging

In [22]:
from nltk import pos_tag

data['POS_tokens'] = data['tokens'].progress_map(lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])
print(data['POS_tokens'].head(1).tolist()[0][:3])


[[('starving_plead', 'JJ'), ('help_runs', 'NNS'), ('wild_video', 'VBP'), ('based', 'VBN')]]


In [23]:
# import spacy
# nlp = spacy.load('en', disable=['parser', 'ner'])

# allowed_postags=['NOUN']

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent))
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])

In [25]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
    
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [26]:
# Lemmatizing each word with its POS tag, in each sentence
data['tokens_sentences_lemmatized'] = data['POS_tokens'].progress_map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

In [27]:
data['tokens_sentences_lemmatized'].head(1).tolist()[0][:3]

[['starving_plead', 'help_runs', 'wild_video', 'base']]

# STTM

In [28]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
data['tokens'] = data['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))

In [31]:
# print("Max number of token:", np.max(data.nb_token))
# print("Mean number of token:", round(np.mean(data.nb_token),2))

# Input format for the model : list of strings (list of tokens)
docs = data['tokens'].tolist()
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)

print("Voc size:", n_terms)
print("Number of documents:", len(docs))

Voc size: 8216
Number of documents: 5657


In [79]:
dictionary_LDA = corpora.Dictionary(docs)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

## Training 

In [67]:
# Train a new model 

# Init of the Gibbs Sampling Dirichlet Mixture Model algorithm
mgp = MovieGroupProcess(K=10, alpha=0.1, beta=0.1, n_iters=100)

vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
n_docs = len(docs)

# Fit the model on the data given the chosen seeds
y = mgp.fit(docs, n_terms)


In stage 0: transferred 4083 clusters with 10 clusters populated
In stage 1: transferred 1843 clusters with 10 clusters populated
In stage 2: transferred 1345 clusters with 10 clusters populated
In stage 3: transferred 1165 clusters with 10 clusters populated
In stage 4: transferred 1012 clusters with 10 clusters populated
In stage 5: transferred 995 clusters with 10 clusters populated
In stage 6: transferred 951 clusters with 10 clusters populated
In stage 7: transferred 933 clusters with 10 clusters populated
In stage 8: transferred 881 clusters with 10 clusters populated
In stage 9: transferred 863 clusters with 10 clusters populated
In stage 10: transferred 911 clusters with 10 clusters populated
In stage 11: transferred 894 clusters with 10 clusters populated
In stage 12: transferred 864 clusters with 10 clusters populated
In stage 13: transferred 877 clusters with 10 clusters populated
In stage 14: transferred 870 clusters with 10 clusters populated
In stage 15: transferred 821 c

In [69]:
# Save model
with open('model_v1.model', "wb") as f:
    pickle.dump(mgp, f)
    f.close()

In [ ]:
# Load the model used in the post
filehandler = open('model_v1.model', 'rb')
mgp = pickle.load(filehandler)

In [71]:
def cluster_importance(mgp):
    n_z_w = mgp.cluster_word_distribution
    beta, V, K = mgp.beta, mgp.vocab_size, mgp.K
    phi = [{} for i in range(K)]
#     print(n_z_w)
    for z in range(K):
        for w in n_z_w[z]:
            phi[z][w] = (n_z_w[z][w]+beta)/(sum(n_z_w[z].values())+V*beta)
    return phi
phi = cluster_importance(mgp)

In [72]:
# print(len(phi))
topic_list = []
for i in range(len(phi)):
    topic_list.append({k: v for k, v in sorted(phi[i].items(), key=lambda item: item[1], reverse=True)})

### Printing Topics

In [74]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topics :', doc_count)
print('*'*20)

# Topics sorted by document inside
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
print('*'*20)

for i in top_index:
    print(colored("Topic #{}".format(i), 'green', attrs=['bold']))
    print(colored("Words: ", 'green', attrs=['bold'])+"[")
    topic_dict = topic_list[i]
    itr = 0
    for key, value in topic_dict.items():
        print("        '"+key+" ("+str(value)+")'")
        itr = itr+1
        if(itr == 20):
            break
    print("]\n\n")

Number of documents per topics : [ 389  626 1033  455  717  369  438  706  584  340]
********************
Most important clusters (by number of docs inside): [2 4 7 1 8 3 6 0 5 9]
********************
Topic #2
Words: [
        'virus (0.013131766615440401)'
        'hospital (0.011836720598138586)'
        'people (0.011405038592371314)'
        'doctor (0.009419301365841865)'
        'outbreak (0.007778909743926234)'
        'chinese (0.007433564139312416)'
        'amp (0.0073472277381589615)'
        'wuhancoronavirus (0.007001882133545145)'
        'spread (0.006052181720857147)'
        'government (0.005965845319703693)'
        'help (0.0056204997150898755)'
        'video (0.005361490511629513)'
        'patient (0.005361490511629513)'
        'wuhanpneumonia (0.004757135703555333)'
        'many (0.004584462901248424)'
        'say (0.004411790098941516)'
        'world (0.004239117296634607)'
        'please (0.004239117296634607)'
        'day (0.004152780895481153)'
       